In [9]:
# import library
# load_data
# split train data & test data
# data normalization
# define class
# define criterion & loss
# define train loop & validation loop

In [10]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np

import matplotlib.pyplot as plt

In [11]:
BATCH_SIZE = 64
NUM_WORKERS = 0

In [12]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(
                                    (0.5,), (0.5,)
                                )])
# Download and load the training & test data
trainset = datasets.MNIST('../data/MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.MNIST('../data/MNIST_data/', download=True, train=False, transform=transform)

In [13]:
# Define loader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
# testset에도 batch를 사용해야 하는가? => ok
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

In [14]:
# get data using iterator
dataiter = iter(trainloader)
images, labels = dataiter.next()

In [15]:
images

tensor([[[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]],


        ...,


        [[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., 

In [16]:
# Define Network
class MLPClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784,512)
        self.fc2 = nn.Linear(512,512)
        self.fc3 = nn.Linear(512,10)

        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = x.view(x.shape[0], -1)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.softmax(self.fc3(x), dim=1)

        return x

In [20]:
model = MLPClassifier()

criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [18]:
epochs = 20
steps = 0

model.train()

for epoch in range(epochs):
    train_loss = 0

    for image, label in trainloader:
        optimizer.zero_grad()

        log_prob = model(image)
        log_loss = criterion(log_prob, label)
        log_loss.backward()
        optimizer.step()

        train_loss += log_loss.item()
    train_loss = train_loss/len(trainloader)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        epoch+1, 
        train_loss
        ))

Epoch: 1 	Training Loss: -0.098933
Epoch: 2 	Training Loss: -0.098714
Epoch: 3 	Training Loss: -0.098747
Epoch: 4 	Training Loss: -0.098747
Epoch: 5 	Training Loss: -0.098764
Epoch: 6 	Training Loss: -0.098697
Epoch: 7 	Training Loss: -0.098714
Epoch: 8 	Training Loss: -0.098714
Epoch: 9 	Training Loss: -0.098697
Epoch: 10 	Training Loss: -0.098714
Epoch: 11 	Training Loss: -0.098714
Epoch: 12 	Training Loss: -0.098714
Epoch: 13 	Training Loss: -0.098697
Epoch: 14 	Training Loss: -0.098714
Epoch: 15 	Training Loss: -0.098731
Epoch: 16 	Training Loss: -0.098731
Epoch: 17 	Training Loss: -0.098781
Epoch: 18 	Training Loss: -0.098731
Epoch: 19 	Training Loss: -0.098731
Epoch: 20 	Training Loss: -0.098731


In [19]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for *evaluation*

for data, target in testloader:
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(BATCH_SIZE):
        label = target[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

IndexError: index 16 is out of bounds for dimension 0 with size 16